Test that the observational condition keywords besides exposure time does not impact the forwardmodel

In [1]:
import os
import numpy as np 
# -- feasibgs -- 
from feasibgs import util as UT
from feasibgs import catalogs as Cat
from feasibgs import forwardmodel as FM

In [2]:
import matplotlib as mpl 
import matplotlib.pyplot as pl 
mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False
%matplotlib inline

In [6]:
cata = Cat.GamaLegacy()
gleg = cata.Read()

In [7]:
redshift = gleg['gama-spec']['z_helio'] # redshift
absmag_ugriz = cata.AbsMag(gleg, kcorr=0.1, Om0=0.3, galext=False) # Absmag k-correct to z = 0.1
ngal = len(redshift)
print('%i galaxies from GAMA-Legacy catalog' % ngal)

39832 galaxies from GAMA-Legacy catalog


/Users/chang/anaconda2/lib/python2.7/site-packages/feasibgs-0.0.0-py2.7.egg/feasibgs/catalogs.py:199: RuntimeWarning: divide by zero encountered in log10
/Users/chang/anaconda2/lib/python2.7/site-packages/feasibgs-0.0.0-py2.7.egg/feasibgs/catalogs.py:199: RuntimeWarning: invalid value encountered in log10


In [9]:
r_mag = UT.flux2mag(gleg['legacy-photo']['apflux_r'][:,1])
vdisp = np.repeat(100., ngal) 

In [8]:
bgs3 = FM.BGStree()
match = bgs3._GamaLegacy(gleg)
hasmatch = (match != -999)
print('%i of the galaxies have matches' % np.sum(hasmatch))

INFO:io.py:1015:read_basis_templates: Reading /Volumes/chang_eHDD/projects/desi/spectro/templates/basis_templates/v2.3/bgs_templates_v2.1.fits metadata.
39702 of the galaxies have matches


/Users/chang/anaconda2/lib/python2.7/site-packages/feasibgs-0.0.0-py2.7.egg/feasibgs/forwardmodel.py:105: RuntimeWarning: invalid value encountered in subtract


In [11]:
n_sub = 100 
in_sub = (hasmatch & (np.arange(ngal) < n_sub))
bgstemp = FM.BGStemplates(wavemin=1500.0, wavemax=2e4)
flux, wave, meta = bgstemp.Spectra(r_mag[in_sub], redshift[in_sub], vdisp[in_sub], seed=1, 
                                   templateid=match[in_sub], silent=False)
wave, flux_eml = bgstemp.addEmissionLines(wave, flux, gleg, np.arange(ngal)[in_sub], silent=False)

INFO:io.py:1027:read_basis_templates: Reading /Volumes/chang_eHDD/projects/desi/spectro/templates/basis_templates/v2.3/bgs_templates_v2.1.fits
DEBUG:templates.py:359:_blurmatrix: Populating blur matrix with 1 unique velocity dispersion values.


In [12]:
fdesi = FM.fakeDESIspec()

Default atmosphere settings: 
- moonalt = -60
- moonsep = 180
- moonfrac = 0.0
- seeing = 1.1

In [26]:
print(flux_eml[0])

[0.01659787 0.01656275 0.01652593 ... 0.24699657 0.24701468 0.24703279]


In [23]:
bgs_spec0 = fdesi.simExposure(wave, flux_eml, skycondition='dark', seed=1, exptime=480)

DEBUG:simexp.py:659:get_source_types: bgs 100 targets


atmosphere setting 1:
- moonalt = 30
- moonsep = 120
- moonfrac = 0.8
- seeing = 1.1

In [22]:
bgs_spec1 = fdesi.simExposure(wave, flux_eml, skycondition='dark', seed=1, exptime=480, 
                              moonalt=30, moonsep=120, moonfrac=0.8, seeing=1.1)

DEBUG:simexp.py:659:get_source_types: bgs 100 targets


In [27]:
print(flux_eml[0])

[0.01659787 0.01656275 0.01652593 ... 0.24699657 0.24701468 0.24703279]


In [31]:
for k in bgs_spec0.flux.keys(): 
    print(k, bgs_spec0.flux[k][0]) 

('r', array([ 2.1103902 ,  2.0180259 , -0.59206986, ..., -0.13992798,
       -1.5267322 , -0.97413677], dtype=float32))
('b', array([ 0.4883322 , -1.1703421 , -2.1506474 , ..., -2.1239378 ,
       -0.4601779 , -0.45549732], dtype=float32))
('z', array([ 3.8521242 , -0.16634664,  4.951483  , ...,  0.5417628 ,
        0.34501553,  0.464745  ], dtype=float32))


In [32]:
for k in bgs_spec0.flux.keys(): 
    print(k, bgs_spec1.flux[k][0]) 

('r', array([ 2.1103902 ,  2.0180259 , -0.59206986, ..., -0.13992798,
       -1.5267322 , -0.97413677], dtype=float32))
('b', array([ 0.4883322 , -1.1703421 , -2.1506474 , ..., -2.1239378 ,
       -0.4601779 , -0.45549732], dtype=float32))
('z', array([ 3.8521242 , -0.16634664,  4.951483  , ...,  0.5417628 ,
        0.34501553,  0.464745  ], dtype=float32))


atmosphere setting 2: 
- moonalt = 90
- moonsep = 20 
- moonfrac = 0.99
- seeing = 1.1

In [29]:
bgs_spec2 = fdesi.simExposure(wave, flux_eml, skycondition='dark', seed=1, exptime=480, 
                              moonalt=90, moonsep=20, moonfrac=0.99, seeing=1.1)

DEBUG:simexp.py:659:get_source_types: bgs 100 targets


In [33]:
for k in bgs_spec0.flux.keys(): 
    print(k, bgs_spec2.flux[k][0]) 

('r', array([ 2.1103902 ,  2.0180259 , -0.59206986, ..., -0.13992798,
       -1.5267322 , -0.97413677], dtype=float32))
('b', array([ 0.4883322 , -1.1703421 , -2.1506474 , ..., -2.1239378 ,
       -0.4601779 , -0.45549732], dtype=float32))
('z', array([ 3.8521242 , -0.16634664,  4.951483  , ...,  0.5417628 ,
        0.34501553,  0.464745  ], dtype=float32))


In [34]:
bgs_spec3 = fdesi.simExposure(wave, flux_eml, skycondition='dark', seed=1, exptime=480, 
                              moonalt=90, moonsep=20, moonfrac=0.99, seeing=1.1, airmass=2.)

DEBUG:simexp.py:659:get_source_types: bgs 100 targets


In [36]:
for k in bgs_spec0.flux.keys(): 
    print(k, bgs_spec3.flux[k][0]) 

('r', array([ 0.41477865,  2.064616  , -1.968128  , ...,  1.4005642 ,
       -0.9474089 ,  2.3228374 ], dtype=float32))
('b', array([ 1.0759289 , -3.886627  ,  6.2750473 , ..., -2.5768106 ,
       -0.31843132,  7.458333  ], dtype=float32))
('z', array([2.7914329 , 2.5405836 , 6.089881  , ..., 0.624763  , 0.6636175 ,
       0.08208259], dtype=float32))


This confirms that the atmosphere keywords (except airmass) do not impact the simulated spectra